In [10]:
import cv2

def save_blink_count_to_file(count):
    with open('blink_count.txt', 'w') as file:
        file.write(str(count))

def load_blink_count_from_file():
    try:
        with open('blink_count.txt', 'r') as file:
            count = int(file.read())
    except FileNotFoundError:
        count = 0
    return count

def detect_eye_blinks():
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    cap = cv2.VideoCapture(0)  # Use the default camera (change the argument for a specific camera index if needed)
    eye_blink_count = load_blink_count_from_file()
    prev_eye_state = False
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
        
        eye_state = False
        for (ex, ey, ew, eh) in eyes:
            eye_roi = gray_frame[ey:ey + eh, ex:ex + ew]
            cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
            
            threshold = 25
            _, threshold_eye_roi = cv2.threshold(eye_roi, threshold, 255, cv2.THRESH_BINARY_INV)
            eye_pixels = cv2.countNonZero(threshold_eye_roi)
            total_pixels = eye_roi.shape[0] * eye_roi.shape[1]
            openness_ratio = eye_pixels / total_pixels
            
            if openness_ratio < 0.2:
                eye_state = True
                
        if prev_eye_state and not eye_state:
            eye_blink_count += 1
            save_blink_count_to_file(eye_blink_count)
            
        prev_eye_state = eye_state
        
        # Display the blink count on the video frame
        cv2.putText(frame, f"Blink Count: {eye_blink_count}", (30, 50), font, font_scale, (0, 0, 255), font_thickness)
        
        cv2.imshow('Shubhankar', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    return eye_blink_count

if __name__ == "__main__":
    blink_count = detect_eye_blinks()
    print("Total eye blinks detected:", blink_count)


Total eye blinks detected: 27
